# Chemistry 313: Machine Learning in Chemistry
Spring 2025, Dinner

In [1]:
import torch
import jax
import numpy as np
import jax.numpy as jnp
import matplotlib.pyplot as plt

In [16]:
jkey = jax.random.PRNGKey(seed=1)
j2key = jax.random.PRNGKey(seed=2)
j3key = jax.random.PRNGKey(seed=3)

## 3/24/25
Types of ML
 - supervised: data = $\{(x_i,y_i)\}_i$, x is features y is label
    - regression ($y \in \mathbb{R}^d$)
    - classifcation ($y \in \mathbb{Z}^d, \{c_0,c_1,\ldots,c_d\}$)
 - unsupervised
    - dimensional reduction (visualization, speed optimization)
    - density estimation
    - clustering
 - RL
    - choosing actions (faces the credit assignment problem which is overdetermined)
    - protein design

test data, train data, validation data (holdout) to prevent overfitting
- in chemistry hard to split because often correlations, "leakage" between train and test
cross validation (kfold)
- make k groups of test-train and cycle through and then average over the groups